In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import transformers
from finetune_peft import get_peft_config, PEFTArguments
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType

/home/ubuntu/minimal-llama/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_path = "/home/ubuntu/llama-weights/7B/llama-7b"
tokenizer_path = model_path

In [4]:
torch.set_default_tensor_type(torch.cuda.HalfTensor)
model = transformers.LlamaForCausalLM.from_pretrained(model_path)

Loading checkpoint shards: 100%|██████████| 33/33 [00:09<00:00,  3.30it/s]


In [19]:
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1
)

model = get_peft_model(model, peft_config)
torch.set_default_tensor_type(torch.cuda.FloatTensor)

tokenizer = transformers.LlamaTokenizer.from_pretrained(tokenizer_path)
batch = tokenizer("The LLaMA language model is", return_tensors="pt")

with torch.no_grad():
    out = model.generate(
        input_ids=batch["input_ids"],
        attention_mask=torch.ones_like(batch["input_ids"]),
        max_length=200,
    )
print(tokenizer.decode(out[0]))

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


 ⁇  The LLaMA language model is a large-scale language model that is trained on a large corpus of text. The LLaMA language model is a large-scale language model that is trained on a large corpus of text.
The LLaMA language model is a large-scale language model that is trained on a large corpus of text. The LLaMA language model is a large-scale language model that is trained on a large corpus of text.
The LLaMA language model is a large-scale language model that is trained on a large corpus of text. The LLaMA language model is a large-scale language model that is trained on a large corpus of text.
The LLaMA language model is a large-scale language model that is trained on a large corpus of text. The LLaMA language model is a large-scale language model that is trained on a large corpus of text.
The LLa


In [8]:
model.base_model.model.model.layers[0].self_attn #.q_proj.lora_A.default.weight

LlamaAttention(
  (q_proj): Linear(
    in_features=4096, out_features=4096, bias=False
    (lora_dropout): ModuleDict(
      (default): Dropout(p=0.1, inplace=False)
    )
    (lora_A): ModuleDict(
      (default): Linear(in_features=4096, out_features=8, bias=False)
    )
    (lora_B): ModuleDict(
      (default): Linear(in_features=8, out_features=4096, bias=False)
    )
    (lora_embedding_A): ParameterDict()
    (lora_embedding_B): ParameterDict()
  )
  (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
  (v_proj): Linear(
    in_features=4096, out_features=4096, bias=False
    (lora_dropout): ModuleDict(
      (default): Dropout(p=0.1, inplace=False)
    )
    (lora_A): ModuleDict(
      (default): Linear(in_features=4096, out_features=8, bias=False)
    )
    (lora_B): ModuleDict(
      (default): Linear(in_features=8, out_features=4096, bias=False)
    )
    (lora_embedding_A): ParameterDict()
    (lora_embedding_B): ParameterDict()
  )
  (o_proj): Linear(in_feat

In [11]:
from peft.tuners.lora import Linear
import torch.nn.functional as F
from peft.utils.other import transpose

In [99]:
bs = 2
seq_len = 8
ctx_dim = 4096
rank = 8
scaling = 4.0
fan_in_fan_out = False
bias = None

x = torch.randn([bs, seq_len, ctx_dim])
weight = torch.randn(([ctx_dim, ctx_dim]))
weight2x = torch.cat([weight, weight], dim=1)

lora_a0 = torch.nn.Linear(in_features=ctx_dim, out_features=rank)
lora_b0 = torch.nn.Linear(in_features=rank, out_features=ctx_dim)

lora_a1 = torch.nn.Linear(in_features=ctx_dim, out_features=rank)
lora_b1 = torch.nn.Linear(in_features=rank, out_features=ctx_dim)

lora_a = torch.nn.Linear(in_features=bs * ctx_dim, out_features=rank, )
lora_b = torch.nn.Linear(in_features=rank, out_features=bs * ctx_dim)

lora_a.weight = torch.nn.Parameter(torch.cat([lora_a0.weight, lora_a1.weight], dim=1))
lora_b.weight = torch.nn.Parameter(torch.cat([lora_b0.weight, lora_b1.weight], dim=0))

# forward pass
result = F.linear(x, transpose(weight, fan_in_fan_out), bias=bias)
x = x.reshape(seq_len, -1)
x = x.to(lora_a.weight.dtype)
out = lora_b(lora_a(x)) * scaling
out = out.reshape(bs, seq_len, -1)
result += out

In [ ]:
def forward(self, x: torch.Tensor):
    previous_dtype = x.dtype
    if self.active_adapter not in self.lora_A.keys():
        return F.linear(x, transpose(self.weight, self.fan_in_fan_out), bias=self.bias)
    if self.disable_adapters:
        if self.r[self.active_adapter] > 0 and self.merged:
            self.unmerge()
        result = F.linear(x, transpose(self.weight, self.fan_in_fan_out), bias=self.bias)
    elif self.r[self.active_adapter] > 0 and not self.merged:
        result = F.linear(x, transpose(self.weight, self.fan_in_fan_out), bias=self.bias)

        x = x.to(self.lora_A[self.active_adapter].weight.dtype)

        result += (
            self.lora_B[self.active_adapter](
                self.lora_A[self.active_adapter](self.lora_dropout[self.active_adapter](x))
            )
            * self.scaling[self.active_adapter]
        )
    else:
        result = F.linear(x, transpose(self.weight, self.fan_in_fan_out), bias=self.bias)

    result = result.to(previous_dtype)

    return result